**ML PROBLEM 4.1**

Alexandre Reis nº100123

Rodolfo Amorim nº100260

In [53]:
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.linear_model import LassoCV, Lasso
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_validate
from sklearn import linear_model 
import matplotlib.pyplot as plt
import numpy as np
from math import *
import os
import warnings
warnings.filterwarnings("ignore")


**Loading the data from the files given**

In [96]:
X_train = np.load(os.path.abspath('X_train_regression1.npy'))
y_train = np.load(os.path.abspath('y_train_regression1.npy'))
X_test  = np.load(os.path.abspath('X_test_regression1.npy'))

**Centering of the data**

In [97]:
X_train_mean  = np.mean(X_train,axis=0) 
y_train_mean  = np.mean(y_train,axis=0)

X_train_prime = X_train.copy()
y_train_prime = y_train.copy()


for i in range(10): #the centering is done with the average of each feature
    X_train_prime[:,i] = X_train_prime[:,i] -  X_train_mean[i]

# y_train_prime is a one dimensional array, and as such we do not include it in the for loop 
y_train_prime = y_train_prime - y_train_mean 



**Application of the LASSO Regularization**
In this case, the objective of using the Lasso Regularization is to create a sparse Beta matrix, which will dictate which of the 10 features of matrix x are important


In [4]:
LassoModelCV = LassoCV(alphas=np.arange(0,10,0.01),cv=4,fit_intercept=False,random_state=0).fit(X_train_prime, y_train_prime) 
#LassoCv is used to determine the best alpha hyperparameter for the lasso model
#we are using cv=8 because it is the one wich proves to give the least mean squared error (this is proven in the final part of this document)
#as the pre-processed data is already centered, fit-intercept is set to false
beta_hat_lasso  = LassoModelCV.coef_ # Beta Coefficients from the LASSO Regularization


# Removal of the unnecessary columns from the X data - this coluns are the pnes corresponding to features with zero value in the Lasso's sparce beta matrix 
X_train_prime_lasso_ridge= X_train_prime.copy()
X_test_lasso_ridge = X_test.copy()
for i in range(np.shape(beta_hat_lasso)[0]-1, -1, -1):
    if beta_hat_lasso[i]==0:
        X_train_prime_lasso_ridge = np.delete(X_train_prime_lasso_ridge, i, 1)
        X_test_lasso_ridge = np.delete(X_test_lasso_ridge, i, 1)
        X_train_mean = np.delete(X_train_mean, i, axis = 0)

**Aplication of the Ridge Regularization to the LASSO Regularization Coefficients**

In [5]:
RidgeModel = linear_model.RidgeCV(alphas=np.arange(0.01,10,0.01), fit_intercept=False, scoring="neg_mean_squared_error", cv=4).fit(X_train_prime_lasso_ridge, y_train_prime)

beta_hat_ridge= RidgeModel.coef_  # Beta Coefficients from the application of the Ridge Regularization to the Coefficients from the LASSO Regularization

**Calculation of the Beta Coefficients to make predictions on Uncentered data**

In [6]:
beta_zero = y_train_mean - np.matmul((X_train_mean),np.transpose(beta_hat_ridge))
m = [beta_zero[0]]
for k in range(((beta_hat_ridge).shape[1])):
        m.append(beta_hat_ridge[:,k][0])
beta_hat_lasso_ridge = np.array(m)

**Prediction for the Test dataset**

In [7]:
X_test_final = np.hstack((np.ones((1000,1)),X_test_lasso_ridge))
prediction = np.matmul(X_test_final,beta_hat_lasso_ridge)
prediciton = prediction.reshape(1000,1)
np.save('prediction_100123_100260_4.1',prediction)

**Appendix**


To prove that the cv used in LassoCv model is the better with regards to the mean squared error and to get to know the optimal range for alpha the following analysis was made:

In [113]:
def MSE_otimization_1(matrix_cv_alpha_score):
    i = np.argmax(matrix_cv_alpha_score[:,2])
    print("The values that optimize the mean squared error are:")
    print("Alpha=" + str(matrix_cv_alpha_score[i,1]))
    print("cv=" + str(matrix_cv_alpha_score[i,0]))
    print("For a MSE=" + str(matrix_cv_alpha_score[i,2]))



In [119]:
def LassoCv_otimization(x,y, fit_intercept_bool):
    LassoOptimization_model_parameters = np.empty([0,3])
    #we are going to test every possible value for k-fold cross validation 
    for cv_value in np.arange(2,15,1):
        LassoOptimization_modelcv = LassoCV(alphas=np.arange(0,10,0.01),cv=cv_value,fit_intercept = fit_intercept_bool, random_state=0).fit(x, y)
      

        cv_results = cross_validate(LassoOptimization_modelcv, x, y,scoring='neg_mean_squared_error', cv=LeaveOneOut())

        neg_mean_squared_errors = cv_results['test_score']
        mean_neg_mean_squared_error = np.mean(neg_mean_squared_errors)

        LassoOptimization_model_parameters = np.vstack((LassoOptimization_model_parameters, np.array([cv_value , LassoOptimization_model.alpha_, mean_neg_mean_squared_error])))
    MSE_otimization_1(LassoOptimization_model_parameters)
    return(LassoOptimization_model_parameters)




In [115]:
print("Otimization for x and y centered train models - Lasso")
Lasso_Optimization_results = LassoCv_otimization(X_train_prime, y_train_prime, False)
Lasso_Optimization_results

Otimization for x and y centered train models - Lasso
The values that optimize the mean squared error are:
Alpha=0.05
cv=4.0
For a MSE=-2.038957005902529


This optimization was done with and without the data previously centered and we concluded that the MSE is closer to zero with centered data

Otimization for x and y centered train models - Lasso
The values that optimize the mean squared error are:
Alpha=0.05
cv=4.0
For a MSE=-2.038957005902529

To prove that the cv used in RidgeCV model is the better with regards to the mean squared error, to get to know the optimal alpha for Ridge's model and to decide wether to center or not the data, the following analysis was made:

In [123]:
def MSE_otimization_2(matrix_cv_alpha_score):
    i = np.argmax(matrix_cv_alpha_score[:,3])
    print("The values that optimize the mean squared error are:")
    print("Alpha=" + str(matrix_cv_alpha_score[i,1]))
    print("cv=" + str(matrix_cv_alpha_score[i,0]))
    print("For a MSE=" + str(matrix_cv_alpha_score[i,3]))
    print("MSE calculated via ridgecv - with fewer coluns in x=" + str(matrix_cv_alpha_score[i,2]))




In [136]:
def RidgeCv_otimization(x,y, fit_intercept_bool):
    RidgeOptimization_model_parameters = np.empty([0,4])
    #we are going to test every possible value for k-fold cross validation including k=15 which is equivalent to the efficient Leave One Out method

    for cv_value in np.arange(2,15,1):
        RidgeOptimization_model = linear_model.RidgeCV(alphas=np.arange(0.01,10,0.01), fit_intercept=fit_intercept_bool, scoring="neg_mean_squared_error", cv=cv_value).fit(x, y)
        
        cv_results = cross_validate(RidgeOptimization_model, x, y,scoring='neg_mean_squared_error', cv=LeaveOneOut())

        neg_mean_squared_errors = cv_results['test_score']
        mean_neg_mean_squared_error = np.mean(neg_mean_squared_errors)

        RidgeOptimization_model_parameters = np.vstack((RidgeOptimization_model_parameters, np.array([cv_value , RidgeOptimization_model.alpha_, RidgeOptimization_model.best_score_, mean_neg_mean_squared_error])))
    MSE_otimization_2(RidgeOptimization_model_parameters)
    return(RidgeOptimization_model_parameters)


In [133]:
def lasso_ridge(x, y):
    Model_lasso = LassoCV(alphas=np.arange(0,10,0.05),cv=4,fit_intercept = False, random_state=0).fit(x, y)
     
    beta_hat_lasso_Model  = Model_lasso.coef_ # Beta Coefficients from the LASSO Regularization


    # Removal of the unnecessary columns from the X data - this coluns are the pnes corresponding to features with zero value in the Lasso's sparce beta matrix 
    X_train_prime_lasso_ridge_appendix= x.copy()
    for i in range(np.shape(beta_hat_lasso_Model)[0]-1, -1, -1):
        if beta_hat_lasso_Model[i]==0:
            X_train_prime_lasso_ridge_appendix = np.delete(X_train_prime_lasso_ridge_appendix, i, 1)

    return X_train_prime_lasso_ridge_appendix

In [137]:
#print("Otimization for x and y original train models")
#RidgeCv_otimization(X_train, y_train, True)

print("Otimization for x and y centered train models")
RidgeCv_otimization(X_train_prime, y_train_prime, False)

print("Otimization for x and y centered train models, with some x coluns/features deleted, based on Lasso's model")

#new x variable that only has coluns for features that Lasso's regularization does not make nul in beta^ matrix
X_train_prime_lasso_ridge_appendix = lasso_ridge(X_train_prime, y_train_prime) #lasso_alpha was defined when optimizing the lasso pareameters

RidgeCv_otimization(X_train_prime_lasso_ridge_appendix, y_train_prime, False)


Otimization for x and y centered train models, with some x coluns/features deleted, based on Lasso's model


KeyboardInterrupt: 

Otimization for x and y centered train models
The values that optimize the mean squared error are:
Alpha=2.1599999999999997
cv=4.0
For a MSE=-2.357425118123902
MSE calculated via ridgecv - with fewer coluns in x=-2.6244248458581385

Otimization for x and y centered train models, with some x coluns/features deleted, based on Lasso's model
The values that optimize the mean squared error are:
Alpha=0.51
cv=4.0
For a MSE=-1.4472918298246262
MSE calculated via ridgecv - with fewer coluns in x=-2.7061605786341976

**Results**


The  best opion is a lasso with cv=4 and alpha=0,05 followed by a ridge with alpha = 0,51 and cv=4 - MSE=-1.4472918298246262

The second best option is a simple lasso with cv=4 and aplha=0,05 - MSE=-2.038957005902529

The thir best option is Ridge model with centered x and y with Alpha=1.31 cv=Leave one out - MSE=-2.167693140920147

